## Generate Weights Verilog Modules from CSV

In [1]:
import os
import math

# --- KONFIGURASI FOLDER & FORMAT ---
FOLDER_A = "data/model_csv/model_1"   # FOLDER INPUT: Tempat file .csv berada
FOLDER_B = "rtl/weights"   # FOLDER OUTPUT: Tempat file .v akan disimpan

DATA_WIDTH = 16    # Lebar bit
SCALE = 1024.0     # Q6.10 (2^10 = 1024)

# Pastikan folder output ada
os.makedirs(FOLDER_A, exist_ok=True)
os.makedirs(FOLDER_B, exist_ok=True)

def to_q6_10(val):
    """Mengubah float ke Hex String Fixed-Point Q6.10 (Two's Complement)"""
    try:
        # Scale & Round
        int_val = int(round(val * SCALE))
        
        # Saturate (Clamp) agar tidak overflow
        max_val = (1 << (DATA_WIDTH - 1)) - 1  # 32767
        min_val = -(1 << (DATA_WIDTH - 1))     # -32768
        int_val = max(min(int_val, max_val), min_val)
        
        # Format ke 4-digit Hex (handle negatif dengan masking)
        return f"{int_val & 0xFFFF:04x}"
    except Exception as e:
        print(f"[Warn] Gagal konversi nilai {val}: {e}")
        return "0000"

def generate_verilog(module_suffix, weight_csv, bias_csv):
    """Membaca sepasang CSV dan menulis file Verilog .v"""
    
    # Path file
    path_w = os.path.join(FOLDER_A, weight_csv)
    path_b = os.path.join(FOLDER_A, bias_csv)
    path_v = os.path.join(FOLDER_B, f"w_{module_suffix}.v")
    
    print(f"Processing: {module_suffix.upper()}...")

    # 1. BACA BIAS
    try:
        with open(path_b, 'r') as f:
            # Asumsi bias adalah satu nilai float
            bias_val = float(f.read().strip())
    except FileNotFoundError:
        print(f"  [Error] Bias file not found: {path_b}")
        return
    except ValueError:
        print(f"  [Error] Invalid data in bias file: {path_b}")
        return

    # 2. BACA WEIGHTS
    try:
        with open(path_w, 'r') as f:
            # Baca semua teks, ganti newline dengan koma (untuk safety), lalu split
            raw_text = f.read().replace('\n', ',')
            # Filter string kosong dan convert ke float
            weights = [float(x) for x in raw_text.split(',') if x.strip()]
    except FileNotFoundError:
        print(f"  [Error] Weight file not found: {path_w}")
        return

    # 3. TULIS VERILOG
    try:
        with open(path_v, 'w') as f:
            f.write(f"module w_{module_suffix} #(parameter DATA_WIDTH={DATA_WIDTH})(\n")
            
            # Deklarasi Output Ports
            ports = [f"output wire signed [DATA_WIDTH-1:0] w{i}" for i in range(len(weights))]
            ports.append("output wire signed [DATA_WIDTH-1:0] bias")
            f.write("    " + ", ".join(ports) + "\n);\n\n")
            
            # Assign Weights
            for i, val in enumerate(weights):
                hex_val = to_q6_10(val)
                f.write(f"    assign w{i} = {DATA_WIDTH}'h{hex_val}; // {val}\n")
            
            # Assign Bias
            hex_bias = to_q6_10(bias_val)
            f.write(f"    assign bias = {DATA_WIDTH}'h{hex_bias}; // {bias_val}\n")
            
            f.write("endmodule\n")
            
        print(f"  -> Generated: {path_v} ({len(weights)} weights)")
        
    except Exception as e:
        print(f"  [Error] Gagal menulis file verilog: {e}")

# --- DAFTAR FILE YANG AKAN DIPROSES ---
# Format: (Suffix Module, Nama File Weight, Nama File Bias)
# Sesuai dengan screenshot file explorer kamu
batch_list = [
    ("enc1", "enc1_weight.csv", "enc1_bias.csv"),
    ("enc2", "enc2_weight.csv", "enc2_bias.csv"),
    ("enc3", "enc3_weight.csv", "enc3_bias.csv"),
    ("dec1", "dec1_weight.csv", "dec1_bias.csv"),
    ("dec2", "dec2_weight.csv", "dec2_bias.csv"),
    ("out",  "out_weight.csv",  "out_bias.csv")
]

# Eksekusi Batch
print(f"--- START CONVERSION (Source: {FOLDER_A} -> Dest: {FOLDER_B}) ---")
for module, w_file, b_file in batch_list:
    generate_verilog(module, w_file, b_file)
print("--- DONE ---")

--- START CONVERSION (Source: data/model_csv/model_1 -> Dest: rtl/weights) ---
Processing: ENC1...
  [Error] Bias file not found: data/model_csv/model_1/enc1_bias.csv
Processing: ENC2...
  [Error] Bias file not found: data/model_csv/model_1/enc2_bias.csv
Processing: ENC3...
  [Error] Bias file not found: data/model_csv/model_1/enc3_bias.csv
Processing: DEC1...
  [Error] Bias file not found: data/model_csv/model_1/dec1_bias.csv
Processing: DEC2...
  [Error] Bias file not found: data/model_csv/model_1/dec2_bias.csv
Processing: OUT...
  [Error] Bias file not found: data/model_csv/model_1/out_bias.csv
--- DONE ---


In [3]:
import numpy as np
import math
import os

# --- KONFIGURASI ---
FOLDER_MEM = "../rtl/memory"
LUT_FILENAME = "tanh_lut.mem"

# Setup parameter sesuai RTL
DATA_WIDTH = 16
SCALE = 1024.0
LUT_DEPTH = 1024

# Parameter Batas RTL (Wajib Match dengan Verilog!)
MIN_IN_VAL = -4096  # Representasi -4.0 dalam Q6.10
# RTL logic: index = (data_in - MIN_IN) >> 3
# Artinya setiap kenaikan 1 index, data_in bertambah 8 poin
STEP_SIZE = 8       

os.makedirs(FOLDER_MEM, exist_ok=True)

def to_q6_10(val):
    """Helper convert float to hex Q6.10"""
    try:
        int_val = int(round(val * SCALE))
        max_val = (1 << (DATA_WIDTH - 1)) - 1
        min_val = -(1 << (DATA_WIDTH - 1))
        int_val = max(min(int_val, max_val), min_val)
        return f"{int_val & 0xFFFF:04x}"
    except:
        return "0000"

def generate_full_range_tanh_lut():
    path_mem = os.path.join(FOLDER_MEM, LUT_FILENAME)
    print(f"Generating Full-Range Tanh LUT: {path_mem} ...")
    
    with open(path_mem, 'w') as f:
        f.write(f"// Tanh LUT Full Range (-4.0 to 4.0)\n")
        f.write(f"// Q6.10 Format. Depth: {LUT_DEPTH}\n")
        
        for i in range(LUT_DEPTH):
            # 1. Hitung Nilai Input X dari Index i
            # Rumus RTL: index = (input - MIN) / 8
            # Balik Rumusnya: input = (index * 8) + MIN
            input_fixed = (i * STEP_SIZE) + MIN_IN_VAL
            
            # Konversi ke float untuk dihitung math.tanh
            x_float = input_fixed / SCALE
            
            # 2. Hitung Output Tanh
            y_float = math.tanh(x_float)
            
            # 3. Konversi ke Hex
            hex_val = to_q6_10(y_float)
            
            f.write(f"{hex_val}\n")
            
            # Debugging (Opsional - print index 0, 512, dan 1023 buat cek bener gak)
            if i == 0:
                print(f"  [Check] Idx 0    (In: {x_float:.2f}) -> Tanh: {y_float:.4f} -> Hex: {hex_val}")
            elif i == 512:
                print(f"  [Check] Idx 512  (In: {x_float:.2f}) -> Tanh: {y_float:.4f} -> Hex: {hex_val}")
            elif i == 1023:
                print(f"  [Check] Idx 1023 (In: {x_float:.2f}) -> Tanh: {y_float:.4f} -> Hex: {hex_val}")

    print(f"  -> Done.")

# --- EKSEKUSI ---
generate_full_range_tanh_lut()

Generating Full-Range Tanh LUT: ../rtl/memory/tanh_lut.mem ...
  [Check] Idx 0    (In: -4.00) -> Tanh: -0.9993 -> Hex: fc01
  [Check] Idx 512  (In: 0.00) -> Tanh: 0.0000 -> Hex: 0000
  [Check] Idx 1023 (In: 3.99) -> Tanh: 0.9993 -> Hex: 03ff
  -> Done.
